In [19]:
import jax
import jax.numpy as jnp
import mujoco
import numpy as np
from mujoco import mjx
from robot_descriptions.z1_mj_description import MJCF_PATH

key = jax.random.PRNGKey(0)

mjmodel = mujoco.MjModel.from_xml_path(MJCF_PATH)
mjdata = mujoco.MjData(mjmodel)

# alter the model so it becomes mjx compatible
mjmodel.dof_frictionloss = 0
mjmodel.opt.integrator = 0

mjxmodel = mjx.put_model(mjmodel)
mjxdata = mjx.put_data(mjmodel, mjdata)

/home/lvjonok/github.com/lvjonok/mujoco-sysid/venv/lib/python3.10/site-packages/jax/_src/interpreters/xla.py:155: RuntimeWarning: overflow encountered in cast
  return np.asarray(x, dtypes.canonicalize_dtype(x.dtype))


In [20]:
q, v = jax.random.normal(key, (2, mjmodel.nq))

q, v

(Array([ 1.1901639 , -1.0996888 ,  0.44367844,  0.5984697 , -0.39189556,
         0.69261974], dtype=float32),
 Array([ 0.46018356, -2.068578  , -0.21438177, -0.9898306 , -0.6789304 ,
         0.27362573], dtype=float32))

In [21]:
mjdata.qpos[:] = q
mjdata.qvel[:] = v

mujoco.mj_step(mjmodel, mjdata)

velocity = np.zeros(6)
for bodyid in mjmodel.jnt_bodyid:
    mujoco.mj_objectVelocity(mjmodel, mjdata, 2, bodyid, velocity, 1)

    print(f"bodyid: {bodyid}, v: {velocity[3:]}, w: {velocity[:3]}")

bodyid: 2, v: [0. 0. 0.], w: [0.         0.         0.46018356]
bodyid: 3, v: [-3.09150672e-18  0.00000000e+00 -1.57468993e-18], w: [ 0.41005399 -2.068578    0.20886511]
bodyid: 4, v: [ 0.31078859 -0.07310279 -0.6539035 ], w: [ 0.28069364 -2.28295977  0.36466421]
bodyid: 5, v: [ 0.23727103 -0.00960553 -0.02728739], w: [ 0.02646465 -3.27279039  0.45942195]
bodyid: 6, v: [0.21066843 0.11146764 0.20180794], w: [ 1.27447096 -3.0145615  -0.21950845]
bodyid: 7, v: [0.21066843 0.30104535 0.20516525], w: [ 1.5480967  -2.46010192  1.75603632]


In [22]:
with np.printoptions(precision=4, suppress=True):
    print(mjdata.cvel)

[[ 0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.4602  0.0068 -0.0015  0.    ]
 [ 1.9205 -0.7685  0.4602  0.0957  0.2207 -0.0307]
 [ 2.1196 -0.8481  0.4602  0.08    0.1816  0.0002]
 [ 3.0386 -1.2159  0.4602  0.0734  0.1651  0.0061]
 [ 3.0531 -1.1796 -0.2176  0.0331  0.1794  0.006 ]
 [ 3.2439 -0.984  -0.2031  0.0367  0.1764 -0.0013]]


In [23]:
from mujoco_sysid.mjx.regressors import object_velocity

mjxdata = mjxdata.replace(qpos=q, qvel=v)
mjxdata = mjx.step(mjxmodel, mjxdata)

In [24]:
with np.printoptions(precision=4, suppress=True):
    print(mjxdata.cvel)

[[ 0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.4602  0.0068 -0.0015  0.    ]
 [ 1.9205 -0.7685  0.4602  0.0957  0.2207 -0.0307]
 [ 2.1196 -0.8481  0.4602  0.08    0.1816  0.0002]
 [ 3.0386 -1.2159  0.4602  0.0734  0.1651  0.0061]
 [ 3.0531 -1.1796 -0.2176  0.0331  0.1794  0.006 ]
 [ 3.2439 -0.984  -0.2031  0.0367  0.1764 -0.0013]]


In [25]:
for bodyid in mjmodel.jnt_bodyid:
    velocity = object_velocity(mjxmodel, mjxdata, bodyid)

    print(f"bodyid: {bodyid}, v: {velocity[3:]}, w: {velocity[:3]}")

bodyid: 2, v: [ 0.00161602 -0.0067537  -0.00010427], w: [0.         0.         0.46018356]
bodyid: 3, v: [ 0.08146539  0.02143904 -0.22914568], w: [ 0.41005418 -2.0685785   0.20886509]
bodyid: 4, v: [ 0.15654099 -0.04573184 -0.1196753 ], w: [ 0.28069377 -2.2829604   0.36466417]
bodyid: 5, v: [ 0.18044616 -0.01005445 -0.00467285], w: [ 0.02646467 -3.2727916   0.45942217]
bodyid: 6, v: [ 0.15331021  0.09907955 -0.00245677], w: [ 1.2744716  -3.0145626  -0.21950865]
bodyid: 7, v: [ 0.15303525  0.06997393 -0.06478949], w: [ 1.5480974 -2.4601028  1.7560369]
